In [1]:
model_name = "nn"

In [2]:
import numpy as np
import pandas as pd
import gc
from time import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, MinMaxScaler

In [3]:
timesheet = [time()]
def timer(statement):
    global timesheet
    timesheet.append(time())
    print statement+" :", (timesheet[-1]-timesheet[-2]),"seconds"

def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## Get Application Train/Test Data

In [4]:
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('../data/application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('../data/application_test.csv', nrows= num_rows)

    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']

    docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
    live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

    inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

    df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY'].astype("float32")
    df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'].astype("float32")
    df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
    df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
    df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN']).astype("float32")
    df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
    df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'].astype("float32")
    df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL']).astype("float32")
    df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
    df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH'].astype("float32")
    df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED'].astype("float32")
    df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH'].astype("float32")
    df['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED'].astype("float32")
    df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'].astype("float32")

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    dropcolum=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 
    'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']
    df= df.drop(dropcolum,axis=1)
    del test_df
    gc.collect()
    return df

## Get Bureau & Bureau Balance Data

In [5]:
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('../data/bureau.csv', nrows = num_rows)
    bb = pd.read_csv('../data/bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': [ 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': [ 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

    num_cat = num_aggregations.copy()
    num_cat.update(cat_aggregations)

    bureau_agg = bureau.groupby('SK_ID_CURR').agg(num_cat)
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    bureau_agg = bureau_agg.reset_index()
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    active_agg = active_agg.reset_index()
    bureau_agg = bureau_agg.merge(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

## Get Previous Applications Data

In [6]:
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('../data/previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': [ 'max', 'mean'],
        'AMT_APPLICATION': [ 'max','mean'],
        'AMT_CREDIT': [ 'max', 'mean'],
        'APP_CREDIT_PERC': [ 'max', 'mean'],
        'AMT_DOWN_PAYMENT': [ 'max', 'mean'],
        'AMT_GOODS_PRICE': [ 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': [ 'max', 'mean'],
        'RATE_DOWN_PAYMENT': [ 'max', 'mean'],
        'DAYS_DECISION': [ 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    num_cat = num_aggregations.copy()
    num_cat.update(cat_aggregations)

    prev_agg = prev.groupby('SK_ID_CURR').agg(num_cat)
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    prev_agg = prev_agg.reset_index()
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    approved_agg = approved_agg.reset_index()
    prev_agg = prev_agg.merge(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

## Get POS Cash Data

In [7]:
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('../data/POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

## Get Installments Payments Data

In [8]:
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('../data/installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum','min','std' ],
        'DBD': ['max', 'mean', 'sum','min','std'],
        'PAYMENT_PERC': [ 'max','mean',  'var','min','std'],
        'PAYMENT_DIFF': [ 'max','mean', 'var','min','std'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum','min','std'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum','std'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum','std']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

## Get Credit Card Balance Data

In [9]:
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('../data/credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg([ 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

# Get Meta Features Generated Before

In [10]:
def get_meta_bb():
    mtr = pd.read_csv("csv/using_lags_bureau_data_train.csv")
    mte = pd.read_csv("csv/using_lags_bureau_data_test.csv")
    data = pd.concat([mtr, mte], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR","BB_META"]
    return data

In [11]:
def get_neptune_bb1():
    mtr = pd.read_csv("../base neptune/m1/csv/lightGBM_out_of_fold_train_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]]
    mte = pd.read_csv("../base neptune/m1/csv/lightGBM_out_of_fold_test_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]].groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    data = pd.concat([mtr, mte], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR","NEPTUNE_META"]
    return data
def get_neptune_bb2():
    mtr = pd.read_csv("../base neptune/m2/csv/lightGBM_out_of_fold_train_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]]
    mte = pd.read_csv("../base neptune/m2/csv/lightGBM_out_of_fold_test_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]].groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    data = pd.concat([mtr, mte], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR","NEPTUNE_META"]
    return data
def get_neptune_bb3():
    mtr = pd.read_csv("../base neptune/m3/csv/lightGBM_out_of_fold_train_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]]
    mte = pd.read_csv("../base neptune/m3/csv/lightGBM_out_of_fold_test_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]].groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    data = pd.concat([mtr, mte], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR","NEPTUNE_META"]
    return data
def get_neptune_bb4():
    mtr = pd.read_csv("../base neptune/m4/csv/lightGBM_out_of_fold_train_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]]
    mte = pd.read_csv("../base neptune/m4/csv/lightGBM_out_of_fold_test_predictions.csv")[["SK_ID_CURR","lightGBM_prediction"]].groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    data = pd.concat([mtr, mte], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR","NEPTUNE_META"]
    return data

# Generate Data For Training

In [12]:
num_rows = None
timer("Init.. ")

data = application_train_test(num_rows)
timer("Done with Application Train and Test")

data = data.merge(bureau_and_balance(num_rows),  how='left', on='SK_ID_CURR')
timer("Done with Bureau and Bureau Balance")
gc.collect()

data = data.merge(previous_applications(num_rows),  how='left', on='SK_ID_CURR')
timer("Done with Previous Applications")
gc.collect()

data = data.join(pos_cash(num_rows),  how='left', on='SK_ID_CURR')
timer("Done with POS Cash")
gc.collect()

data = data.join(installments_payments(num_rows),  how='left', on='SK_ID_CURR')
timer("Done with Installment Payments")
gc.collect()

data = data.join(credit_card_balance(num_rows),  how='left', on='SK_ID_CURR')
timer("Done with Credit Card balance")
gc.collect()

data = data.merge(get_meta_bb(),  how='left', on='SK_ID_CURR')
timer("Done with BB Meta Features")
gc.collect()

data = data.merge(get_neptune_bb1(),  how='left', on='SK_ID_CURR')
data = data.merge(get_neptune_bb2(),  how='left', on='SK_ID_CURR')
data = data.merge(get_neptune_bb3(),  how='left', on='SK_ID_CURR')
data = data.merge(get_neptune_bb4(),  how='left', on='SK_ID_CURR')
timer("Done with NEPTUNE Meta Features")
gc.collect()

# Rescaling
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna(axis = 1, how = "all")
gc.collect()

for c in data.columns:
    if c not in ["TARGET","SK_ID_CURR"]:
        scaled = MinMaxScaler().fit_transform(data[c].astype("float32").fillna(data[c].mean()).values.reshape(-1, 1))
        data[c] = pd.Series(scaled.flatten())
gc.collect()

train = data[data['TARGET'].notnull()]
test = data[data['TARGET'].isnull()]
del data
gc.collect()

target = train.TARGET
train_id = train[["SK_ID_CURR"]]
test_id = test[["SK_ID_CURR"]]

train = train.drop(['TARGET','SK_ID_CURR','index'], axis=1)
test = test.drop(['TARGET','SK_ID_CURR','index'], axis=1)

Init..  : 2.43877506256 seconds
Train samples: 307511, test samples: 48744
Done with Application Train and Test : 17.3194758892 seconds
Done with Bureau and Bureau Balance : 41.9092540741 seconds
Done with Previous Applications : 35.1088039875 seconds
Done with POS Cash : 23.6520528793 seconds
Done with Installment Payments : 64.2859530449 seconds
Done with Credit Card balance : 27.4315900803 seconds
Done with BB Meta Features : 1.836414814 seconds
Done with NEPTUNE Meta Features : 7.46669006348 seconds


# Defining Model

In [13]:
np.random.seed(0)
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Activation,InputLayer, Dropout, PReLU, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras import metrics
from keras import optimizers
from keras import backend as K

K.tf.set_random_seed(0)
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True 
K.set_session(K.tf.Session(config=cfg))

def model_nn(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id):
    model = Sequential()
    model.add(Dense(units = 10 , kernel_initializer = 'normal', input_dim=x_train.shape[1]))
    model.add(PReLU())
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer="rmsprop")
    
    checkpoint = ModelCheckpoint("weights/{}.hdf5".format(model_name), monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='auto') 

    model.fit(x_train, y_train, epochs=100, batch_size=500, validation_data =(x_test, y_test), callbacks=[checkpoint, early_stop], class_weight={1:np.mean(y_train), 0: 1-np.mean(y_train)})

    model.load_weights("weights/{}.hdf5".format(model_name))
    meta_train[test_index] = model.predict(x_test, batch_size=32).flatten()
    meta_test.append(model.predict(test, batch_size=32).flatten())
    print "ROC_AUC_SCORE: {}".format(roc_auc_score(y_test, meta_train[test_index]))

    del model
    gc.collect()
    K.clear_session()
    K.tf.set_random_seed(0)

/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Training Classifier

In [14]:
meta_train = np.zeros(train.shape[0])
meta_test = []

kf = StratifiedKFold(n_splits= 10, shuffle=True, random_state=47)
for fold_id, (train_index, test_index) in enumerate(kf.split(train, target)):
    x_train, x_test = train.iloc[train_index].as_matrix(), train.iloc[test_index].as_matrix()
    y_train, y_test = target[train_index], target[test_index]

    model_nn(x_train, x_test, y_train, y_test, test.as_matrix(), meta_train, meta_test,train_index, test_index,fold_id)

print "OVERALL ROC_AUC_SCORE: {}".format(roc_auc_score(target, meta_train))
test_id["TARGET"] = np.array(meta_test).T.mean(axis=1)
train_id["TARGET"] = meta_train

Train on 276755 samples, validate on 30752 samples
Epoch 1/100
275000/276755 [============================>.] - ETA: 0s - loss: 0.0618
Epoch 00001: val_loss improved from inf to 0.37326, saving model to weights/nn.hdf5
276755/276755 [==============================] - 11s 38us/step - loss: 0.0616 - val_loss: 0.3733
Epoch 2/100
275000/276755 [============================>.] - ETA: 0s - loss: 0.0355
Epoch 00002: val_loss improved from 0.37326 to 0.36218, saving model to weights/nn.hdf5
276755/276755 [==============================] - 9s 34us/step - loss: 0.0355 - val_loss: 0.3622
Epoch 3/100
275000/276755 [============================>.] - ETA: 0s - loss: 0.0351
Epoch 00003: val_loss did not improve
276755/276755 [==============================] - 10s 35us/step - loss: 0.0351 - val_loss: 0.3809
Epoch 4/100
276000/276755 [============================>.] - ETA: 0s - loss: 0.0350
Epoch 00004: val_loss did not improve
276755/276755 [==============================] - 10s 36us/step - loss: 0.03

Epoch 3/100
276500/276756 [============================>.] - ETA: 0s - loss: 0.0351
Epoch 00003: val_loss improved from 0.36587 to 0.36006, saving model to weights/nn.hdf5
276756/276756 [==============================] - 10s 36us/step - loss: 0.0351 - val_loss: 0.3601
Epoch 4/100
275500/276756 [============================>.] - ETA: 0s - loss: 0.0350
Epoch 00004: val_loss did not improve
276756/276756 [==============================] - 9s 34us/step - loss: 0.0350 - val_loss: 0.3793
Epoch 5/100
276000/276756 [============================>.] - ETA: 0s - loss: 0.0350
Epoch 00005: val_loss improved from 0.36006 to 0.35653, saving model to weights/nn.hdf5
276756/276756 [==============================] - 10s 35us/step - loss: 0.0350 - val_loss: 0.3565
Epoch 6/100
276000/276756 [============================>.] - ETA: 0s - loss: 0.0350
Epoch 00006: val_loss improved from 0.35653 to 0.34869, saving model to weights/nn.hdf5
276756/276756 [==============================] - 10s 35us/step - loss: 0

Epoch 6/100
276500/276757 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00006: val_loss did not improve
276757/276757 [==============================] - 10s 36us/step - loss: 0.0349 - val_loss: 0.3653
Epoch 7/100
276000/276757 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00007: val_loss did not improve
276757/276757 [==============================] - 10s 37us/step - loss: 0.0349 - val_loss: 0.3567
Epoch 8/100
275000/276757 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00008: val_loss did not improve
276757/276757 [==============================] - 9s 34us/step - loss: 0.0349 - val_loss: 0.3558
Epoch 9/100
276000/276757 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00009: val_loss did not improve
276757/276757 [==============================] - 9s 34us/step - loss: 0.0349 - val_loss: 0.3790
ROC_AUC_SCORE: 0.754057906612
Train on 276757 samples, validate on 30750 samples
Epoch 1/100
276500/276757 [==================

Epoch 3/100
276500/276757 [============================>.] - ETA: 0s - loss: 0.0352
Epoch 00003: val_loss did not improve
276757/276757 [==============================] - 9s 34us/step - loss: 0.0352 - val_loss: 0.3650
Epoch 4/100
276000/276757 [============================>.] - ETA: 0s - loss: 0.0351
Epoch 00004: val_loss did not improve
276757/276757 [==============================] - 10s 35us/step - loss: 0.0350 - val_loss: 0.3702
Epoch 5/100
275500/276757 [============================>.] - ETA: 0s - loss: 0.0350
Epoch 00005: val_loss improved from 0.35914 to 0.35401, saving model to weights/nn.hdf5
276757/276757 [==============================] - 9s 34us/step - loss: 0.0350 - val_loss: 0.3540
Epoch 6/100
276500/276757 [============================>.] - ETA: 0s - loss: 0.0349
Epoch 00006: val_loss did not improve
276757/276757 [==============================] - 9s 34us/step - loss: 0.0349 - val_loss: 0.3546
Epoch 7/100
276500/276757 [============================>.] - ETA: 0s - loss: 

In [16]:
train_id.to_csv("csv/{}_train.csv".format(model_name), index=False)
test_id.to_csv("csv/{}_test.csv".format(model_name), index=False)